In [1]:
import os
import yfinance as yf
from pathlib import Path
import itertools
import yfinance as yf
import os
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from fredapi import Fred

In [ ]:

PROJECT_DIR = Path.cwd()
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
OUT_DIR = os.path.join(PROJECT_DIR, 'output')
for d in [DATA_DIR, OUT_DIR]:
    os.makedirs(d, exist_ok=True)

In [ ]:


DATA_DIR = "data"


TICKERS = [
    '^GSPC',  # S&P 500
    '^VIX',   # VIX Index
    'GC=F',   # Gold Futures
    'CL=F'    # Crude Oil Futures
]


START = "2001-01-02"

END = "2025-11-14" 


os.makedirs(DATA_DIR, exist_ok=True)

for t in TICKERS:
    print(f'  -> Téléchargement de {t}')
    
    
    df = yf.download(t, 
                     start=START, 
                     end=END, 
                     progress=False,
                     auto_adjust=False)
    
   
    
    
    out = df.reset_index()
    
   
    safe_ticker_name = t.replace('^', '_')
    file_path = os.path.join(DATA_DIR, f'{safe_ticker_name}.csv')
    
    
    out.to_csv(file_path, index=False)





Dossier de données assuré : data/
Début du téléchargement...
  -> Téléchargement de ^GSPC
  -> Téléchargement de ^VIX
  -> Téléchargement de GC=F
  -> Téléchargement de CL=F
--- Téléchargement terminé. ---


In [ ]:
import pandas as pd
import os


DATA_DIR = "data"

files = {
    'sp500': '_GSPC.csv',
    'vix': '_VIX.csv',
    'gold': 'GC=F.csv',
    'oil': 'CL=F.csv'
}



df_sp500 = pd.read_csv(
    os.path.join(DATA_DIR, files['sp500']),
    parse_dates=['Date'],
    index_col='Date'
)

df_vix = pd.read_csv(
    os.path.join(DATA_DIR, files['vix']),
    parse_dates=['Date'],
    index_col='Date'
)

df_gold = pd.read_csv(
    os.path.join(DATA_DIR, files['gold']),
    parse_dates=['Date'],
    index_col='Date'
)

df_oil = pd.read_csv(
    os.path.join(DATA_DIR, files['oil']),
    parse_dates=['Date'],
    index_col='Date'
)




Chargement des 4 ensembles de données...
Données brutes chargées avec succès.


In [5]:
#Dropping the first useless line with thicker indication

df_sp500 = df_sp500.iloc[1:]
df_vix    = df_vix.iloc[1:]
df_gold   = df_gold.iloc[1:]
df_oil    = df_oil.iloc[1:]

In [6]:
print(df_sp500.isna().sum())

Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64


In [7]:
print(df_oil.isna().sum())

Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64


In [8]:
print(df_gold.isna().sum())


Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64


In [9]:
print(df_vix.isna().sum())


Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64


In [ ]:
#Agregating in one dataframe

master_df = df_sp500.join(df_vix, how='outer', rsuffix='_VIX')
master_df = master_df.join(df_gold, how='outer', rsuffix='_GOLD')
master_df = master_df.join(df_oil, how='outer', rsuffix='_OIL')

#Date sorting
master_df.sort_index(inplace=True)



Jointure des DataFrames...


In [12]:
print(master_df.isna().sum())

Adj Close          5
Close              5
High               5
Low                5
Open               5
Volume             5
Adj Close_VIX      5
Close_VIX          5
High_VIX           5
Low_VIX            5
Open_VIX           5
Volume_VIX         5
Adj Close_GOLD    18
Close_GOLD        18
High_GOLD         18
Low_GOLD          18
Open_GOLD         18
Volume_GOLD       18
Adj Close_OIL     14
Close_OIL         14
High_OIL          14
Low_OIL           14
Open_OIL          14
Volume_OIL        14
dtype: int64


In [ ]:
# Forward fill for Nanhandling
master_df.fillna(method='ffill', inplace=True)

# Erase first nan that cannot be forward filled
master_df.dropna(inplace=True)



C:\Users\rvent\AppData\Local\Temp\ipykernel_20012\1525924657.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  master_df.fillna(method='ffill', inplace=True)


In [ ]:
#Drop collumn that we do not want:

columns_to_drop = [
    'Volume_VIX',   
    'Volume_GOLD',  
    'Volume_OIL'   
]



master_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')


In [15]:
master_df.head()

,Adj Close,Close,High,Low,Open,Volume,Adj Close_VIX,Close_VIX,High_VIX,Low_VIX,...,Adj Close_GOLD,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Adj Close_OIL,Close_OIL,High_OIL,Low_OIL,Open_OIL
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.27001953125,1283.27001953125,1320.280029296875,1276.050048828125,1320.280029296875,1129400000,29.989999771118164,29.989999771118164,30.399999618530273,27.920000076293945,...,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,27.200000762939453,27.200000762939453,27.399999618530273,26.600000381469727,27.25
2001-01-03,1347.56005859375,1347.56005859375,1347.760009765625,1274.6199951171875,1283.27001953125,1880700000,26.600000381469727,26.600000381469727,29.959999084472656,25.989999771118164,...,268.0,268.0,268.0,268.0,268.0,27.950000762939453,27.950000762939453,28.139999389648438,27.049999237060547,27.229999542236328
2001-01-04,1333.3399658203125,1333.3399658203125,1350.239990234375,1329.1400146484375,1347.56005859375,2131000000,26.969999313354492,26.969999313354492,27.190000534057617,26.1299991607666,...,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,28.200000762939453,28.200000762939453,28.780000686645508,27.850000381469727,28.200000762939453
2001-01-05,1298.3499755859375,1298.3499755859375,1334.77001953125,1294.949951171875,1333.3399658203125,1430800000,28.670000076293945,28.670000076293945,29.040000915527344,27.209999084472656,...,268.0,268.0,268.0,268.0,268.0,28.0,28.0,28.799999237060547,27.799999237060547,28.149999618530273
2001-01-08,1295.8599853515625,1295.8599853515625,1298.3499755859375,1276.2900390625,1298.3499755859375,1115500000,29.84000015258789,29.84000015258789,30.799999237060547,29.719999313354492,...,268.0,268.0,268.0,268.0,268.0,27.350000381469727,27.350000381469727,28.399999618530273,27.149999618530273,28.200000762939453


In [ ]:

print(list(master_df.columns))

#  Define the base S&P 500 columns to rename 

columns_to_rename = ['Open', 'Close', 'Adj Close', 'High', 'Low', 'Volume']

#  Create the renaming dictionary 

rename_map = {col: f"{col}_S&P" for col in columns_to_rename}

#Apply the renaming to the DataFrame
master_df.rename(columns=rename_map, inplace=True)

print(list(master_df.columns))

--- Columns BEFORE renaming ---
['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'Adj Close_VIX', 'Close_VIX', 'High_VIX', 'Low_VIX', 'Open_VIX', 'Adj Close_GOLD', 'Close_GOLD', 'High_GOLD', 'Low_GOLD', 'Open_GOLD', 'Adj Close_OIL', 'Close_OIL', 'High_OIL', 'Low_OIL', 'Open_OIL']

--- Columns AFTER renaming ---
['Adj Close_S&P', 'Close_S&P', 'High_S&P', 'Low_S&P', 'Open_S&P', 'Volume_S&P', 'Adj Close_VIX', 'Close_VIX', 'High_VIX', 'Low_VIX', 'Open_VIX', 'Adj Close_GOLD', 'Close_GOLD', 'High_GOLD', 'Low_GOLD', 'Open_GOLD', 'Adj Close_OIL', 'Close_OIL', 'High_OIL', 'Low_OIL', 'Open_OIL']


In [ ]:
#Checking for identical collumns:

identical_pairs = []

# Get all unique combinations of 2 column names
all_pairs = itertools.combinations(master_df.columns, 2)

# Loop through each pair and check if they are equal
for col_a, col_b in all_pairs:
    if master_df[col_a].equals(master_df[col_b]):
        identical_pairs.append((col_a, col_b))

#  Report the results 
if not identical_pairs:
    print("\nResult: No identical columns were found.")
else:
    print("\nResult: Found the following identical columns:")
    for pair in identical_pairs:
        print(f"  - '{pair[0]}' is identical to '{pair[1]}'")


Result: Found the following identical columns:
  - 'Adj Close_S&P' is identical to 'Close_S&P'
  - 'Adj Close_VIX' is identical to 'Close_VIX'
  - 'Adj Close_GOLD' is identical to 'Close_GOLD'
  - 'Adj Close_OIL' is identical to 'Close_OIL'


In [ ]:
#Drop the identivcal collumns:

print(list(master_df.columns))



columns_to_drop = [
    'Adj Close_S&P',
    'Adj Close_VIX',
    'Adj Close_GOLD',
    'Adj Close_OIL'
]


master_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')


print(list(master_df.columns))


['Adj Close_S&P', 'Close_S&P', 'High_S&P', 'Low_S&P', 'Open_S&P', 'Volume_S&P', 'Adj Close_VIX', 'Close_VIX', 'High_VIX', 'Low_VIX', 'Open_VIX', 'Adj Close_GOLD', 'Close_GOLD', 'High_GOLD', 'Low_GOLD', 'Open_GOLD', 'Adj Close_OIL', 'Close_OIL', 'High_OIL', 'Low_OIL', 'Open_OIL']

--- Columns AFTER dropping ---
['Close_S&P', 'High_S&P', 'Low_S&P', 'Open_S&P', 'Volume_S&P', 'Close_VIX', 'High_VIX', 'Low_VIX', 'Open_VIX', 'Close_GOLD', 'High_GOLD', 'Low_GOLD', 'Open_GOLD', 'Close_OIL', 'High_OIL', 'Low_OIL', 'Open_OIL']


In [19]:
master_df.head()

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL
Date,,,,,,,,,,,,,,,,,
2001-01-02,1283.27001953125,1320.280029296875,1276.050048828125,1320.280029296875,1129400000,29.989999771118164,30.399999618530273,27.920000076293945,27.93000030517578,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,27.200000762939453,27.399999618530273,26.600000381469727,27.25
2001-01-03,1347.56005859375,1347.760009765625,1274.6199951171875,1283.27001953125,1880700000,26.600000381469727,29.959999084472656,25.989999771118164,29.6299991607666,268.0,268.0,268.0,268.0,27.950000762939453,28.139999389648438,27.049999237060547,27.229999542236328
2001-01-04,1333.3399658203125,1350.239990234375,1329.1400146484375,1347.56005859375,2131000000,26.969999313354492,27.190000534057617,26.1299991607666,26.350000381469727,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,28.200000762939453,28.780000686645508,27.850000381469727,28.200000762939453
2001-01-05,1298.3499755859375,1334.77001953125,1294.949951171875,1333.3399658203125,1430800000,28.670000076293945,29.040000915527344,27.209999084472656,27.809999465942383,268.0,268.0,268.0,268.0,28.0,28.799999237060547,27.799999237060547,28.149999618530273
2001-01-08,1295.8599853515625,1298.3499755859375,1276.2900390625,1298.3499755859375,1115500000,29.84000015258789,30.799999237060547,29.719999313354492,29.979999542236328,268.0,268.0,268.0,268.0,27.350000381469727,28.399999618530273,27.149999618530273,28.200000762939453


In [ ]:
#Intraday realised volatility
var = pd.read_csv("data/oxfordmanrealizedvolatilityindices.csv")

In [21]:
var.head()

,Unnamed: 0,Symbol,close_price,open_to_close,medrv,rsv,rv5,rv10_ss,rv5_ss,bv_ss,rk_twoscale,open_price,bv,open_time,rv10,rk_parzen,close_time,rk_th2,rsv_ss,nobs
0,2000-01-03 00:00:00+00:00,.AEX,675.44,-0.000340,0.000050,0.000046,0.000130,0.000178,0.000130,0.000100,0.000103,675.67,0.000100,90101.0,0.000178,0.000179,163015.0,0.000102,0.000046,1795.0
1,2000-01-04 00:00:00+00:00,.AEX,642.25,-0.033606,0.000075,0.000147,0.000201,0.000261,0.000201,0.000207,0.000199,664.20,0.000207,90416.0,0.000261,0.000423,163016.0,0.000201,0.000147,1785.0
2,2000-01-05 00:00:00+00:00,.AEX,632.31,-0.001675,0.000166,0.000328,0.000491,0.000714,0.000491,0.000361,0.000325,633.37,0.000361,90016.0,0.000714,0.000324,163016.0,0.000345,0.000328,1801.0
3,2000-01-06 00:00:00+00:00,.AEX,624.21,-0.013130,0.000152,0.000116,0.000225,0.000182,0.000225,0.000258,0.000218,632.46,0.000258,90016.0,0.000182,0.000219,163002.0,0.000221,0.000116,1799.0
4,2000-01-07 00:00:00+00:00,.AEX,644.86,0.025013,0.000039,0.000048,0.000138,0.000157,0.000138,0.000130,0.000126,628.93,0.000130,90046.0,0.000157,0.000155,163016.0,0.000123,0.000048,1798.0


In [22]:
#Check the different value in the collumn Symbol:  
var['Symbol'].unique()

array(['.AEX', '.AORD', '.BFX', '.BSESN', '.BVLG', '.BVSP', '.DJI',
       '.FCHI', '.FTMIB', '.FTSE', '.GDAXI', '.GSPTSE', '.HSI', '.IBEX',
       '.IXIC', '.KS11', '.KSE', '.MXX', '.N225', '.NSEI', '.OMXC20',
       '.OMXHPI', '.OMXSPI', '.OSEAX', '.RUT', '.SMSI', '.SPX', '.SSEC',
       '.SSMI', '.STI', '.STOXX50E'], dtype=object)

In [23]:
#Rename date collumn
var.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
#Drop all the rows that are not .SPX in the collumn Symbol
var_SPX = var[var['Symbol'] == '.SPX'].copy()
#Drop the Symbol collumn as it is not useful anymore
var_SPX.drop(columns=['Symbol'], inplace=True)


In [24]:
#Keep the collumn we need:

#Keep Date,  rv5 rv5_ss columns
var_SPX = var_SPX[['Date', 'rv5', 'rv5_ss']]


In [25]:
var_SPX.head()

,Date,rv5,rv5_ss
125189,2000-01-03 00:00:00+00:00,0.000141,0.000141
125190,2000-01-04 00:00:00+00:00,0.000224,0.000224
125191,2000-01-05 00:00:00+00:00,0.000314,0.000314
125192,2000-01-06 00:00:00+00:00,0.000131,0.000131
125193,2000-01-07 00:00:00+00:00,0.000094,0.000094


In [26]:
#Changing the date collumn to have the right format

#Change the date format as a string
var_SPX['Date'] = var_SPX['Date'].astype(str)

#Keep only the part we are interested in
var_SPX['Date'] = var_SPX['Date'].str.split(' ').str[0]

#Convert Date collum to datetime format
var_SPX['Date'] = pd.to_datetime(var_SPX['Date'], format='%Y-%m-%d')

#Set the date as index:
var_SPX = var_SPX.set_index('Date')

In [27]:
var_SPX.head()

,rv5,rv5_ss
Date,,
2000-01-03,0.000141,0.000141
2000-01-04,0.000224,0.000224
2000-01-05,0.000314,0.000314
2000-01-06,0.000131,0.000131
2000-01-07,0.000094,0.000094


In [28]:
#Check if there is some Nans:  
print(var_SPX.isna().sum())

rv5       0
rv5_ss    0
dtype: int64


In [29]:
merged = master_df.join(var_SPX, how='inner')

In [30]:
merged

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,rv5_ss
Date,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.27001953125,1320.280029296875,1276.050048828125,1320.280029296875,1129400000,29.989999771118164,30.399999618530273,27.920000076293945,27.93000030517578,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,27.200000762939453,27.399999618530273,26.600000381469727,27.25,0.000203,0.000203
2001-01-03,1347.56005859375,1347.760009765625,1274.6199951171875,1283.27001953125,1880700000,26.600000381469727,29.959999084472656,25.989999771118164,29.6299991607666,268.0,268.0,268.0,268.0,27.950000762939453,28.139999389648438,27.049999237060547,27.229999542236328,0.001003,0.001003
2001-01-04,1333.3399658203125,1350.239990234375,1329.1400146484375,1347.56005859375,2131000000,26.969999313354492,27.190000534057617,26.1299991607666,26.350000381469727,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,28.200000762939453,28.780000686645508,27.850000381469727,28.200000762939453,0.000159,0.000159
2001-01-05,1298.3499755859375,1334.77001953125,1294.949951171875,1333.3399658203125,1430800000,28.670000076293945,29.040000915527344,27.209999084472656,27.809999465942383,268.0,268.0,268.0,268.0,28.0,28.799999237060547,27.799999237060547,28.149999618530273,0.000244,0.000244
2001-01-08,1295.8599853515625,1298.3499755859375,1276.2900390625,1298.3499755859375,1115500000,29.84000015258789,30.799999237060547,29.719999313354492,29.979999542236328,268.0,268.0,268.0,268.0,27.350000381469727,28.399999618530273,27.149999618530273,28.200000762939453,0.000083,0.000083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10,3909.8798828125,3931.5,3884.93994140625,3920.780029296875,4837070000,21.989999771118164,23.850000381469727,19.690000534057617,21.639999389648438,1840.5999755859375,1851.0999755859375,1837.0,1842.5999755859375,58.68000030517578,58.90999984741211,58.08000183105469,58.45000076293945,0.000078,0.000078
2021-02-11,3916.3798828125,3925.989990234375,3890.389892578125,3916.39990234375,4590960000,21.25,23.25,21.110000610351562,22.09000015258789,1824.9000244140625,1845.0,1820.9000244140625,1840.5,58.2400016784668,58.709999084472656,57.84000015258789,58.400001525878906,0.000043,0.000043
2021-02-12,3934.830078125,3937.22998046875,3905.780029296875,3911.64990234375,4135060000,19.969999313354492,22.450000762939453,19.950000762939453,21.600000381469727,1821.5999755859375,1827.5,1809.199951171875,1825.0,59.470001220703125,59.81999969482422,57.40999984741211,57.939998626708984,0.000015,0.000015


In [31]:
merged.index.min()


Timestamp('2001-01-02 00:00:00')

In [32]:
merged.index.max()

Timestamp('2021-02-17 00:00:00')

In [33]:
#Now let's get the 10y treasury rate from FRED, using fred API


fred = Fred(api_key='e45a95bdc314815747f423df1713c9bb')

dgs10 = fred.get_series('DGS10', observation_start='2001-01-02', observation_end='2021-02-17')

In [34]:
dgs10 = dgs10.to_frame(name='DGS10')

In [35]:
dgs10.shape

(5252, 1)

In [36]:
dgs10

,DGS10
2001-01-02,4.92
2001-01-03,5.14
2001-01-04,5.03
2001-01-05,4.93
2001-01-08,4.94
...,...
2021-02-11,1.16
2021-02-12,1.20
2021-02-15,NaN
2021-02-16,1.30


In [37]:
merged = merged.join(dgs10, how='left')


In [38]:
merged.head()

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,rv5_ss,DGS10
Date,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.27001953125,1320.280029296875,1276.050048828125,1320.280029296875,1129400000,29.989999771118164,30.399999618530273,27.920000076293945,27.93000030517578,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,27.200000762939453,27.399999618530273,26.600000381469727,27.25,0.000203,0.000203,4.92
2001-01-03,1347.56005859375,1347.760009765625,1274.6199951171875,1283.27001953125,1880700000,26.600000381469727,29.959999084472656,25.989999771118164,29.6299991607666,268.0,268.0,268.0,268.0,27.950000762939453,28.139999389648438,27.049999237060547,27.229999542236328,0.001003,0.001003,5.14
2001-01-04,1333.3399658203125,1350.239990234375,1329.1400146484375,1347.56005859375,2131000000,26.969999313354492,27.190000534057617,26.1299991607666,26.350000381469727,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,28.200000762939453,28.780000686645508,27.850000381469727,28.200000762939453,0.000159,0.000159,5.03
2001-01-05,1298.3499755859375,1334.77001953125,1294.949951171875,1333.3399658203125,1430800000,28.670000076293945,29.040000915527344,27.209999084472656,27.809999465942383,268.0,268.0,268.0,268.0,28.0,28.799999237060547,27.799999237060547,28.149999618530273,0.000244,0.000244,4.93
2001-01-08,1295.8599853515625,1298.3499755859375,1276.2900390625,1298.3499755859375,1115500000,29.84000015258789,30.799999237060547,29.719999313354492,29.979999542236328,268.0,268.0,268.0,268.0,27.350000381469727,28.399999618530273,27.149999618530273,28.200000762939453,0.000083,0.000083,4.94


In [39]:
#Convert GDS10 collumn to percentage:

merged['DGS10'] = merged['DGS10'] / 100

In [40]:
merged.head()

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,rv5_ss,DGS10
Date,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.27001953125,1320.280029296875,1276.050048828125,1320.280029296875,1129400000,29.989999771118164,30.399999618530273,27.920000076293945,27.93000030517578,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,27.200000762939453,27.399999618530273,26.600000381469727,27.25,0.000203,0.000203,0.0492
2001-01-03,1347.56005859375,1347.760009765625,1274.6199951171875,1283.27001953125,1880700000,26.600000381469727,29.959999084472656,25.989999771118164,29.6299991607666,268.0,268.0,268.0,268.0,27.950000762939453,28.139999389648438,27.049999237060547,27.229999542236328,0.001003,0.001003,0.0514
2001-01-04,1333.3399658203125,1350.239990234375,1329.1400146484375,1347.56005859375,2131000000,26.969999313354492,27.190000534057617,26.1299991607666,26.350000381469727,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,28.200000762939453,28.780000686645508,27.850000381469727,28.200000762939453,0.000159,0.000159,0.0503
2001-01-05,1298.3499755859375,1334.77001953125,1294.949951171875,1333.3399658203125,1430800000,28.670000076293945,29.040000915527344,27.209999084472656,27.809999465942383,268.0,268.0,268.0,268.0,28.0,28.799999237060547,27.799999237060547,28.149999618530273,0.000244,0.000244,0.0493
2001-01-08,1295.8599853515625,1298.3499755859375,1276.2900390625,1298.3499755859375,1115500000,29.84000015258789,30.799999237060547,29.719999313354492,29.979999542236328,268.0,268.0,268.0,268.0,27.350000381469727,28.399999618530273,27.149999618530273,28.200000762939453,0.000083,0.000083,0.0494


In [41]:
print(merged.isna().sum())

Close_S&P      0
High_S&P       0
Low_S&P        0
Open_S&P       0
Volume_S&P     0
Close_VIX      0
High_VIX       0
Low_VIX        0
Open_VIX       0
Close_GOLD     0
High_GOLD      0
Low_GOLD       0
Open_GOLD      0
Close_OIL      0
High_OIL       0
Low_OIL        0
Open_OIL       0
rv5            0
rv5_ss         0
DGS10         37
dtype: int64


In [42]:
#US equity market and the US treadury markekt do not share the same holiday calendar, we will use forward fill to fill the missing values in DGS10
merged['DGS10'].fillna(method='ffill', inplace=True)

C:\Users\rvent\AppData\Local\Temp\ipykernel_20012\1348264306.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged['DGS10'].fillna(method='ffill', inplace=True)
C:\Users\rvent\AppData\Local\Temp\ipykernel_20012\1348264306.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged['DGS10'].fillna(method='ffill', inplace=True)


In [43]:
print(merged.isna().sum())

Close_S&P     0
High_S&P      0
Low_S&P       0
Open_S&P      0
Volume_S&P    0
Close_VIX     0
High_VIX      0
Low_VIX       0
Open_VIX      0
Close_GOLD    0
High_GOLD     0
Low_GOLD      0
Open_GOLD     0
Close_OIL     0
High_OIL      0
Low_OIL       0
Open_OIL      0
rv5           0
rv5_ss        0
DGS10         0
dtype: int64


In [44]:
merged.head()

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,rv5_ss,DGS10
Date,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.27001953125,1320.280029296875,1276.050048828125,1320.280029296875,1129400000,29.989999771118164,30.399999618530273,27.920000076293945,27.93000030517578,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,27.200000762939453,27.399999618530273,26.600000381469727,27.25,0.000203,0.000203,0.0492
2001-01-03,1347.56005859375,1347.760009765625,1274.6199951171875,1283.27001953125,1880700000,26.600000381469727,29.959999084472656,25.989999771118164,29.6299991607666,268.0,268.0,268.0,268.0,27.950000762939453,28.139999389648438,27.049999237060547,27.229999542236328,0.001003,0.001003,0.0514
2001-01-04,1333.3399658203125,1350.239990234375,1329.1400146484375,1347.56005859375,2131000000,26.969999313354492,27.190000534057617,26.1299991607666,26.350000381469727,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,28.200000762939453,28.780000686645508,27.850000381469727,28.200000762939453,0.000159,0.000159,0.0503
2001-01-05,1298.3499755859375,1334.77001953125,1294.949951171875,1333.3399658203125,1430800000,28.670000076293945,29.040000915527344,27.209999084472656,27.809999465942383,268.0,268.0,268.0,268.0,28.0,28.799999237060547,27.799999237060547,28.149999618530273,0.000244,0.000244,0.0493
2001-01-08,1295.8599853515625,1298.3499755859375,1276.2900390625,1298.3499755859375,1115500000,29.84000015258789,30.799999237060547,29.719999313354492,29.979999542236328,268.0,268.0,268.0,268.0,27.350000381469727,28.399999618530273,27.149999618530273,28.200000762939453,0.000083,0.000083,0.0494


In [45]:
#Show me the collumn list of the merged dataframe:
print(list(merged.columns))

['Close_S&P', 'High_S&P', 'Low_S&P', 'Open_S&P', 'Volume_S&P', 'Close_VIX', 'High_VIX', 'Low_VIX', 'Open_VIX', 'Close_GOLD', 'High_GOLD', 'Low_GOLD', 'Open_GOLD', 'Close_OIL', 'High_OIL', 'Low_OIL', 'Open_OIL', 'rv5', 'rv5_ss', 'DGS10']


In [ ]:
#NOW FOCUSING ON BULDING THE TARGET VARIABLE: 
#Here we want to take into account the intraday realized volatility (catpured by our High frequency estimator): 

#Transforming realized intraday variance into volatility
merged["rv5"] = np.sqrt(merged["rv5"])



In [47]:
#keep only the basic 5 minut estimator for now:
merged= merged.drop(columns=["rv5_ss"])

In [48]:
merged.head()

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,DGS10
Date,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.27001953125,1320.280029296875,1276.050048828125,1320.280029296875,1129400000,29.989999771118164,30.399999618530273,27.920000076293945,27.93000030517578,268.3999938964844,268.3999938964844,268.3999938964844,268.3999938964844,27.200000762939453,27.399999618530273,26.600000381469727,27.25,0.014231,0.0492
2001-01-03,1347.56005859375,1347.760009765625,1274.6199951171875,1283.27001953125,1880700000,26.600000381469727,29.959999084472656,25.989999771118164,29.6299991607666,268.0,268.0,268.0,268.0,27.950000762939453,28.139999389648438,27.049999237060547,27.229999542236328,0.031663,0.0514
2001-01-04,1333.3399658203125,1350.239990234375,1329.1400146484375,1347.56005859375,2131000000,26.969999313354492,27.190000534057617,26.1299991607666,26.350000381469727,267.29998779296875,267.29998779296875,267.29998779296875,267.29998779296875,28.200000762939453,28.780000686645508,27.850000381469727,28.200000762939453,0.012610,0.0503
2001-01-05,1298.3499755859375,1334.77001953125,1294.949951171875,1333.3399658203125,1430800000,28.670000076293945,29.040000915527344,27.209999084472656,27.809999465942383,268.0,268.0,268.0,268.0,28.0,28.799999237060547,27.799999237060547,28.149999618530273,0.015626,0.0493
2001-01-08,1295.8599853515625,1298.3499755859375,1276.2900390625,1298.3499755859375,1115500000,29.84000015258789,30.799999237060547,29.719999313354492,29.979999542236328,268.0,268.0,268.0,268.0,27.350000381469727,28.399999618530273,27.149999618530273,28.200000762939453,0.009083,0.0494


In [49]:
#Ensure all collumns are numeric:
merged = merged.apply(pd.to_numeric)

In [50]:
#Create the log-returns collumn:
merged["ret"] = np.log(merged["Close_S&P"]).diff()
#Drop the first raw and reset the index:


In [51]:
merged

,Close_S&P,High_S&P,Low_S&P,Open_S&P,Volume_S&P,Close_VIX,High_VIX,Low_VIX,Open_VIX,Close_GOLD,High_GOLD,Low_GOLD,Open_GOLD,Close_OIL,High_OIL,Low_OIL,Open_OIL,rv5,DGS10,ret
Date,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.270020,1320.280029,1276.050049,1320.280029,1129400000,29.990000,30.400000,27.920000,27.930000,268.399994,268.399994,268.399994,268.399994,27.200001,27.400000,26.600000,27.250000,0.014231,0.0492,NaN
2001-01-03,1347.560059,1347.760010,1274.619995,1283.270020,1880700000,26.600000,29.959999,25.990000,29.629999,268.000000,268.000000,268.000000,268.000000,27.950001,28.139999,27.049999,27.230000,0.031663,0.0514,0.048884
2001-01-04,1333.339966,1350.239990,1329.140015,1347.560059,2131000000,26.969999,27.190001,26.129999,26.350000,267.299988,267.299988,267.299988,267.299988,28.200001,28.780001,27.850000,28.200001,0.012610,0.0503,-0.010609
2001-01-05,1298.349976,1334.770020,1294.949951,1333.339966,1430800000,28.670000,29.040001,27.209999,27.809999,268.000000,268.000000,268.000000,268.000000,28.000000,28.799999,27.799999,28.150000,0.015626,0.0493,-0.026593
2001-01-08,1295.859985,1298.349976,1276.290039,1298.349976,1115500000,29.840000,30.799999,29.719999,29.980000,268.000000,268.000000,268.000000,268.000000,27.350000,28.400000,27.150000,28.200001,0.009083,0.0494,-0.001920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10,3909.879883,3931.500000,3884.939941,3920.780029,4837070000,21.990000,23.850000,19.690001,21.639999,1840.599976,1851.099976,1837.000000,1842.599976,58.680000,58.910000,58.080002,58.450001,0.008828,0.0115,-0.000345
2021-02-11,3916.379883,3925.989990,3890.389893,3916.399902,4590960000,21.250000,23.250000,21.110001,22.090000,1824.900024,1845.000000,1820.900024,1840.500000,58.240002,58.709999,57.840000,58.400002,0.006555,0.0116,0.001661
2021-02-12,3934.830078,3937.229980,3905.780029,3911.649902,4135060000,19.969999,22.450001,19.950001,21.600000,1821.599976,1827.500000,1809.199951,1825.000000,59.470001,59.820000,57.410000,57.939999,0.003810,0.0120,0.004700


In [52]:

# Save the dataframe to CSV

merged.to_csv(os.path.join(DATA_DIR, 'merged_data.csv'), index=True)